In [21]:
import pickle
import tarfile
from random import choices
from math import inf
from itertools import product, cycle
import numpy as np
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm

In [39]:
np.random.seed(321)

# Downloading the dataset

In [40]:
def unpickle(file):
    """
    source: https://www.cs.toronto.edu/~kriz/cifar.html
    """
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [41]:
dataset = requests.get('https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz')

with open('cifar-10-python.tar.gz', 'wb') as dataset_file:
    dataset_file.write(dataset.content)

In [42]:
with tarfile.open('cifar-10-python.tar.gz') as tar:
    tar.extractall()

# "Training" the model or preprocessing the dataset

In [43]:
def process_image(data: np.ndarray) -> np.ndarray:
    """
    
    takes image as a row of 3072 elements, 
    where the first 1024 entries contain the red channel values,
    the next 1024 the green, and the final 1024 the blue.
    
    returns image as an array of shape (32, 32, 3)
    
    """
    red_channel, green_channel, blue_channel = np.split(data, 3)
    pixels = np.stack((red_channel, green_channel, blue_channel), axis=-1)
    return np.reshape(pixels, (32, 32, 3))

In [44]:
def process_batch(batch: np.ndarray) -> tuple[list[np.ndarray], list[int]]:
    """
    
    processes a batch and returns a tuple of a list of processed images with shape (32, 32, 3) and list of labels
    
    """
    batch_images = list(map(process_image, batch[b'data']))
    batch_labels = batch[b'labels']
    
    return batch_images, batch_labels

In [45]:
batch_filenames = ['cifar-10-batches-py/data_batch_1',
                   'cifar-10-batches-py/data_batch_2',
                   'cifar-10-batches-py/data_batch_3', 
                   'cifar-10-batches-py/data_batch_4',
                   'cifar-10-batches-py/data_batch_5']

train_images, raw_train_labels = [], []
for filename in batch_filenames:
    current_batch = unpickle(filename)
    images, labels = process_batch(current_batch)
    train_images.extend(images)
    raw_train_labels.extend(labels)

In [46]:
def one_hot_encoder(labels: list[int]) -> list[np.ndarray]:
    encoded: list[np.ndarray] = []
    for label in labels:
        encoded.append(np.zeros(10))
        encoded[-1][label] = 1.0
    return encoded

In [47]:
train_labels = one_hot_encoder(raw_train_labels)

In [48]:
test_batch = unpickle('cifar-10-batches-py/test_batch')
test_images, raw_test_labels = process_batch(test_batch)

In [49]:
test_labels = one_hot_encoder(raw_test_labels)

In [50]:
meta_data = unpickle('cifar-10-batches-py/batches.meta')
label_names = list(map(bytes.decode, meta_data[b'label_names']))

In [51]:
def model(x: np.ndarray, weights: np.ndarray) -> np.ndarray:
    res = np.matmul(weights, x.reshape((-1, 1))) 
    return res / np.max(res)

In [52]:
def softmax(x):
    exp = np.exp(x)
    return exp / np.sum(exp)


def loss(predicted, true):
    return -np.dot(true, np.log(predicted))

In [53]:
best_weights = None
min_loss = inf

for i in tqdm(range(100)):
    w = np.random.randn(10, 32 * 32 * 3)
    current_losses = []
    for label, image in zip(train_labels, train_images):
        prediction = softmax(model(image, w))
        current_losses.append(loss(prediction, label))
    mean_loss = np.mean(current_losses)
    if min_loss > mean_loss:
        best_weights = w
        min_loss = mean_loss

print(f'minimal loss: {min_loss}')

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\peppa\AppData\Local\Temp\ipykernel_9692\459627731.py:2: RuntimeWarning: overflow encountered in exp
  exp = np.exp(x)
C:\Users\peppa\AppData\Local\Temp\ipykernel_9692\459627731.py:3: RuntimeWarning: invalid value encountered in divide
  return exp / np.sum(exp)
C:\Users\peppa\AppData\Local\Temp\ipykernel_9692\459627731.py:7: RuntimeWarning: divide by zero encountered in log
  return -np.dot(true, np.log(predicted))
100%|██████████| 100/100 [08:01<00:00,  4.82s/it]

minimal loss: 2.370082459785033


In [54]:
predicted_labels = []
true_labels = []

for label, image in zip(test_labels, test_images):
    y = np.matmul(best_weights, image.reshape((-1, 1))) 
    predicted_labels.append(np.argmax(y))
    true_labels.append(np.argmax(label))
accuracy = sum([int(predicted == true) for predicted, true in zip(predicted_labels, true_labels)]) / len(predicted_labels)

In [55]:
accuracy

0.1103